In [2]:
# -*- encoding:utf-8 -*-
import pandas as pd
import base64
import urllib.parse
import re
import nltk
import urllib
import numpy as np
from sklearn import cross_validation
from sklearn import tree
from sklearn.grid_search import GridSearchCV

In [60]:
def is_number(str):
    try:
        float(str)
        return True
    except:
        return False

def is_letter(str):
    return str.isalpha() and len(str) <=2

def is_base64_code(str):
    if len(str) < 20 :
        return False
    return  re.match('^[a-zA-Z0-9+-/=_]*[a-zA-Z0-9+-/=_]$', str)# re.match('[a-zA-Z0-9+/=]', str)

def is_base64_code_ex(str):
    if len(str) < 20:
        return False
    return  re.match('^[a-zA-Z0-9+-/=_ ]*[a-zA-Z0-9+-/=_ ]$', str)

def base64_url_decode(inp):
    # 通过url传输时去掉了=号，所以需要补上=号
    if len(inp) % 4 != 0:
        try:
            return base64.urlsafe_b64decode(str(inp)+'='* (4 - len(inp) % 4))
        except:
            return inp
    else:
        try:
            return base64.urlsafe_b64decode(str(inp))
        except:
            return inp

def base64_url_encode(inp):
    return base64.urlsafe_b64encode(str(inp)).rstrip('=')

def to_bytestring(s, enc='utf-8'):
    if s:
        if isinstance(s, str):
            return s
        else :
            return s.encode(enc)

def decode_file(file_name):
    names = ['no', 'flag', 'url', 'action']
    df = pd.read_csv(file_name, sep = ',', encoding = 'utf-8', names=names)
    params_list = []
    for no, flag, action in zip(df['no'], df['flag'], df['action']):
        new_params = []
        if type(action) is float:
            #print (no, 'dddd')
            action=str(action)
            params_list.append(action)
            continue
        rs = urllib.parse.urlparse('http://xxx?'+action)

        query_set = urllib.parse.parse_qs(rs.query)
        if len(query_set) == 0:
            #print action
            new_params.append(action)
            params_list.append(new_params)
            continue
        for key, value in query_set.items():
            param = ''.join(value)

            if is_base64_code(param):
                new_param = base64_url_decode(param)
            else:
                new_param = ''
                if is_base64_code_ex(param) :
                    try:
                        for part_param in param.split(' '):
                            try:
                                temp = base64_url_decode(part_param)
                                new_param += str(temp)
                            except:
                                new_param += part_param
                    except:
                        new_param = param
                else :
                    new_param = param

            new_params.append(new_param)
            query_set[key] = new_param.split()
        params_list.append(new_params)
        flag_list=[]
        for item in df['flag']:
            if item=="w":
                flag_list.append(1)
            else:
                flag_list.append(0)
    return params_list,flag_list
def cut_word(params_list):
    doc_word_list = []
    for params in params_list:
        param_word_list = list()
        text = nltk.word_tokenize(str(params))
        for w in text:
            w = w.lower().lstrip(' ').rstrip(' ').replace("u'", ' ')
            if w[0] == "'":
                w = w[1:]
            w = w.lstrip(' ').rstrip(' ')
            w=w.lstrip('[').rstrip(']')
            if (is_number(w) == False and len(w) < 50 and is_letter(w) == False and len(w) != 0):
                param_word_list.append(w)
        doc_word_list.append(param_word_list)

    return doc_word_list


In [61]:
def kmp(word_list,feature):
    cnt=0
    for word in word_list:
        if word in feature:
            cnt=cnt+1
    return cnt

In [62]:
def get_doc_features_matrix(doc_word_list, features):
    rows = len(doc_word_list)
    cols = len(features)
    doc_features_matrix = np.zeros((rows, cols))
    for i in range(1, rows):
        for j in range(1, cols):
            doc_features_matrix[i][j] = kmp(doc_word_list[i],features[j])*len(features[j])
    return doc_features_matrix

In [63]:
file_name = 'F:\contest\subject2_sample\subject2_sample.txt'
params_list,y= decode_file(file_name)
doc_word_list = cut_word(params_list)

In [64]:
features=["fclose","script_name","filestools","query","rewriterule","rewritecond","charset=","multipart/form-data","/wwwroot/dwyshw11/wwwroot/home.php","-|","windows-1251","@","/home2/risquemag/public_html/","php_self","charset=utf-8","host_name.","content_mb","--","die","host_name","execut","/wwwroot/dnwcomcn//jiaozhuo.php","rewritebase","domain",";","public","netstat","[","posix_getegid","ini_set","{","index.php","substr","&","document_root","doctype","rewriterule","root","tar","strlen","127.0.0.1","!","end","php_uname","ping","foreach","xhtml","-an","http","description","urldecode","enctype","charset=gb2312","opts","delfile","echo","php_config.php","remote_server.",".substr","fwrite","set_magic_quotes_runtime","<",".php","eval","set_time_limit","execute","gethttppage","ifmodule","filesman","|","meta","loginlogin","print","eval","wscript","http-equiv=","gb2312","xmlns=","echo","get_current_user","array","mkdir","windows","white","google|aol|yahoo|msn|search|bing|seznam|seznam","filesize","function_exists","shell","jstephens","para","fires","path_translated","robots.txt","_p'.'os","melissacalhoun","ie=edge","path_translated","and","response.write","content_mb=gethttppage","^","phpfunc","-","=","-zcvf","begin","content=","net","mod_rewrite.c","content_mb=file_get_contents","buf=","dom","url","}","keywords","mkfile","parameters","title","(","163.com","_server","get","server_name","query_st","transitional",".htaccess","view","posix_getpwuid","set","is_dir",'#',"wordpress","url=","c1sall","jquery.js","base64_decode","php.exe","remote_server","ifmodule","query_string","http","login","dirname","script_filename","chmod","reg_type","array_map","usr","cmd",">","add","host","main","downdb","rename","http_user_agent","systeminfo","base64_","index.php",")","prc","html","http","set_time_limit","user","function","/*","str_replace","rewritecond","post","base64_decode","touch","date_default_timezone_set","$","location","utf-8","php_self","request_filename","web.config","fires","text/html","header","edit","rewriterule","posix_geteuid","hunter","header","for","http_host","?","run","isset","range","content-type","visible","base64_decode","fopen","rewritecond",".*","display_errors","]","*","wordpress",":","head","pwd","eval","buf",'index.php","edoced_46esab',"rewriteengine","/bin/sh","trim","fread",'%','http_referer','x-ua-compatible',"str_replace",'eval',"downfile"]
doc_features_matrix = get_doc_features_matrix(doc_word_list, features)

In [65]:
#print (doc_word_list)

In [80]:
X=doc_features_matrix
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1, random_state=35)

In [81]:
clf=tree.DecisionTreeClassifier(max_depth=15,min_samples_split=2,min_samples_leaf=1,max_features=140)
clf.fit(X_train,y_train)
scores =cross_validation .cross_val_score(clf, X_train, y_train, cv=10, scoring='f1')
# #clf.feature_importances_
print (scores.mean())

0.847511267333


In [77]:
clf=tree.DecisionTreeClassifier(random_state=35)
parameters = {'max_depth': [12,13,14,15,16],
             'min_samples_split': [2,3,4],  
            'min_samples_leaf': [1, 2, 3,4,5] 
             ,'max_features':[100,]}
grid = GridSearchCV(clf, parameters, 'f1', cv=10)
grid = grid.fit(X_train, y_train)
reg = grid.best_estimator_
print('best score: %f'%grid.best_score_)
print('best parameters:')
for key in parameters.keys():
    print('%s: %d'%(key, reg.get_params()[key]))
print('test score: %f'%reg.score(X_test, y_test))


best score: 0.856694
best parameters:
max_depth: 16
min_samples_split: 2
min_samples_leaf: 1
max_features: 100
test score: 0.918919
